In [28]:
using StatsBase, DrWatson, LinearAlgebra, Random

function weight_index(i::Int64,j::Int64,k::Int64; Nx::Int64 = 1, Ny::Int64 = 1)
    return i + Nx*(j-1) + Nx*Ny*(k-1)
end

function new_sim_param(DT::Float64, v0::Float64, DR::Float64, N::Int64, L::Int64, ϕa::Float64, ϕp::Float64; T::Float64 = 0.001, name::String = "test", save_interval::Float64 = 0.001, save_on::Bool = false)
    param::Dict{String, Any} = Dict{String,Any}()
    N₁::Int64,N₂::Int64 = L*N, N
    @pack! param = DT, v0, DR, N, L, ϕa, ϕp, T , name, N₁, N₂, save_interval, save_on
    return param
end

function get_jump_rate(η::Array{Float64, 3}, x₁::Int64, x₂::Int64, y₁::Int64, y₂::Int64, jump::Int64, DT::Float64, v0::Float64, N::Int64, N₁::Int64,N₂::Int64)
    is_valid::Int64 = η[x₁, x₂,1]*(1-η[y₁, y₂,1])
    bias::Float64 = 0.
    if jump == 2 #left
        bias  = -η[x₁, x₂,2]
    elseif jump == 3 #right
        bias  = η[x₁, x₂,2]
    end
    return is_valid*( DT*N^2 + v0*bias*N ) 
end

function initiate_uniform(ϕa::Float64, ϕp::Float64, N₁,N₂)
    η::Array{Float64, 3} = zeros(N₁,N₂,2)
    # x₁, x₂, spin = i, j , k
    w::Weights{Float64, Float64, Vector{Float64}} = weights([ϕa/2, ϕa/2, ϕp, (1 - ϕa - ϕp)])
    particle::Vector{Vector{Int64}} = [ [1,1], [1,-1], [1,0], [0,0]]
    for x₁ in 1:N₁, x₂ in 1:N₂
        η[x₁, x₂, :] = sample(particle,w)
    end
    return η
end

function initiate_weights(η::Array{Float64, 3}, N₁::Int64, N₂::Int64, DT::Float64, v0::Float64, DR::Float64, N::Int64)
    w::Array{Float64, 3} = zeros(N₁,N₂,5)
    jumps::Vector{Tuple{Int64, Int64}} = [(i,j) for i in -1:1:1, j in -1:1:1 if i^2+j^2 ==1 ];
    for x₁ in 1:N₁, x₂ in 1:N₂, jump in 1:4
        local y₁::Int64, y₂::Int64
        # find adjacent site
        y₁, y₂ = ( (x₁, x₂) .+ jumps[jump] .+ (N₁-1, N₂-1) ).% (N₁, N₂) .+ (1,1)
        w[x₁, x₂, jump] =  get_jump_rate(η, x₁, x₂, y₁, y₂, jump, DT, v0, N, N₁,N₂)
    end
    for x₁ in 1:N₁, x₂ in 1:N₂
        local jump::Int64
        jump = 5
        w[x₁, x₂, jump] = DR*η[x₁, x₂,2]^2
    end
    return weights(w)
end

function model_step!(η::Array{Float64, 3}, w::Weights{Float64, Float64, Vector{Float64}}, t::Float64, N₁::Int64, N₂::Int64, DT::Float64, v0::Float64, N::Int64, jumps::Vector{Tuple{Int64,Int64}}) 
    #update total propensity
    prop::Float64 = sum(w)
    #update time
    t += randexp()/prop
    #select jump
    x::Array{Tuple{Int64, Int64, Int64}, 3} = [ (i,j,k) for i in 1:N₁, j in 1:N₂, k in 1:5]
    x₁::Int64, x₂::Int64, jump::Int64 =  sample(x,w)
    if jump == 5
        #execute jump
        η[x₁, x₂, 2] = - η[x₁, x₂, 2]
        #update rates
        for jump in 1:4
            local y₁::Int64, y₂::Int64
            # find adjacent site
            y₁, y₂ = ( (x₁, x₂) .+ jumps[jump] .+ (N₁-1, N₂-1) ).% (N₁, N₂) .+ (1,1)
            w[weight_index(x₁, x₂,jump; Nx =N₁, Ny =N₂)] = get_jump_rate(η, x₁, x₂,y₁, y₂, jump, DT, v0, N, N₁,N₂)
        end
    else
        local y₁::Int64, y₂::Int64
        # find adjacent site
        y₁, y₂ = ( (x₁, x₂) .+ jumps[jump] .+ (N₁-1, N₂-1) ).% (N₁, N₂) .+ (1,1)
        # swap particles
        η[x₁, x₂, :], η[y₁, y₂, :] = η[y₁, y₂, :], η[x₁, x₂, :]
        # set hop rates
        for jump in 1:4
            local z₁::Int64, z₂::Int64 
            #update adjacent sites to x
            z₁, z₂ = ( (x₁, x₂) .+ jumps[jump] .+ (N₁-1, N₂-1) ).% (N₁, N₂) .+ (1,1)
            w[weight_index(x₁, x₂,  jump; Nx =N₁, Ny =N₂)] = 0. # as x is empty
            w[weight_index(z₁, z₂,5-jump; Nx =N₁, Ny =N₂)] = get_jump_rate(η, z₁, z₂, x₁, x₂, 5-jump, DT, v0, N, N₁,N₂)
        
            #update adjacent sites to y
            z₁, z₂ = ( (y₁, y₂) .+ jumps[jump] .+ (N₁-1, N₂-1) ).% (N₁, N₂) .+ (1,1)
            w[weight_index(y₁, y₂,  jump; Nx =N₁, Ny =N₂)] = get_jump_rate(η, y₁, y₂, z₁, z₂,   jump, DT, v0, N, N₁,N₂)
            w[weight_index(z₁, z₂,5-jump; Nx =N₁, Ny =N₂)] = get_jump_rate(η, z₁, z₂, y₁, y₂, 5-jump, DT, v0, N, N₁,N₂)
        end
        # set flip rates
        w[weight_index(x₁, x₂, 5; Nx =N₁, Ny =N₂)] = 0.
        w[weight_index(y₁, y₂, 5; Nx =N₁, Ny =N₂)] = DR*η[y₁, y₂, 2]^2
    end
    return t
end

function sim_save_name(param::Dict{String, Any},t::Float64)
    @unpack DT, v0, DR, N, L, ϕa, ϕp, name, save_interval = param
    s = round(t ; digits = Int64( -log10(save_interval) ÷ 1 ))
    return "/store/DAMTP/jm2386/Active_Lattice/data/sims_raw/$(name)/t=$(s)_[DT,v0,DR,N,L,ϕa,ϕp]=$([DT,v0,DR,N,L,ϕa,ϕp]).jld2"
end

function run_new_sim(param::Dict{String, Any})
    @unpack DT, v0, DR, N, L, ϕa, ϕp, T , name, N₁, N₂, save_interval, save_on = param
    # configuration
    η::Array{Float64, 3} = initiate_uniform(ϕa, ϕp, N₁, N₂,);
    w::Weights{Float64, Float64, Vector{Float64}} = initiate_weights(η, N₁, N₂, DT, v0, DR,N); 
    t::Float64 = 0.;
    s::Float64 = save_interval

    #inital save
    if save_on
        filename::String        = sim_save_name(param,t)
        data::Dict{String, Any} = Dict("η" => η, "t" => t)
        safesave(filename,data)
    end

    while t < T
        while t < s
            t = model_step!(η, w, t, N₁, N₂, DT, v0, N, jumps);
        end
        #save snapshot
        if save_on
            filename    = sim_save_name(param,t)
            data        = Dict("η" => η, "t" => t)
            safesave(filename,data)
        end
        s += save_interval
    end
    return η, w, t
end

function run_current_sim(param::Dict{String, Any},dt::Float64, η::Array{Float64, 3}, w::Weights{Float64, Float64, Vector{Float64}}, t::Float64)
    @unpack DT, v0, DR, N, L, ϕa, ϕp , name, N₁, N₂, save_interval, save_on = param
    # configuration
    s::Float64 = t + save_interval

    #inital save
    if save_on
        filename::String = sim_save_name(param,t)
        data::Dict{String, Any} = Dict("η" => η, "t" => t)
        safesave(filename,data)
    end

    while t < t+dt
        while t < s
            t = model_step!(η, w, t, N₁, N₂, DT, v0, N, jumps);
        end
        #save snapshot
        if save_on
            filename = sim_save_name(param,t)
            data = Dict("η" => η, "t" => t)
            safesave(filename,data)
        end
        s += save_interval
    end
    return η, w, t
end


run_current_sim (generic function with 1 method)

In [25]:
# Parameters
DT, v0, DR, N, L, ϕa, ϕp = (1.0, 7.5, 1.0, 10, 2, 0.5, 0.0);
sim_param = new_sim_param(DT, v0, DR, N, L, ϕa, ϕp; T = 0.001, name = "test", save_interval = 0.001);
@unpack DT, v0, DR, N, L, ϕa, ϕp, T , name, N₁, N₂ = sim_param

# configuration
η = initiate_uniform(ϕa, ϕp, N₁, N₂,);
w = initiate_weights(η, N₁, N₂, DT, v0, DR,N); 
t =0.;

jumps = [(i,j) for i in -1:1:1, j in -1:1:1 if i^2+j^2 ==1 ];

t = model_step!(η, w, t, N₁, N₂, DT, v0, N, jumps);

sim_save_name(sim_param,t)

"/store/DAMTP/jm2386/Active_Lattice/data/sims_raw/test/t=0.0_[DT,v0,DR,N,L,ϕa,ϕp]=[1.0, 7.5, 1.0, 10.0, 2.0, 0.5, 0.0].jld2"

In [30]:

DT, v0, DR, N, L, ϕa, ϕp = (1.0, 7.5, 1.0, 10, 2, 0.5, 0.0);
sim_param = new_sim_param(DT, v0, DR, N, L, ϕa, ϕp; T = 0.01, name = "test", save_interval = 0.001, save_on = true);
η, w, t = run_new_sim(sim_param)

([0.0 1.0 … 1.0 0.0; 0.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 0.0 … 1.0 0.0;;; 0.0 -1.0 … -1.0 0.0; 0.0 -1.0 … -1.0 1.0; … ; -1.0 1.0 … 1.0 1.0; -1.0 0.0 … -1.0 0.0], [0.0, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0], 0.010041046751674576)